In [1]:
import pandas as pd
import numpy as np

ww = pd.read_csv('C:\Asus WebStorage\psabin@gmail.com\MySyncFolder\Data Science Course\BlueBerry Winery\winequality-white.csv', sep=';')
rw = pd.read_csv('C:\Asus WebStorage\psabin@gmail.com\MySyncFolder\Data Science Course\BlueBerry Winery\winequality-red.csv', sep=';')

In [2]:
ww['quality_label'] = ww['quality'].apply(lambda value: 'low'
if value <= 5 else 'medium'
if value <= 7 else 'high')

# here we are transforming these labels into categrical data type (specific to pandas) instead of simple string
ww['quality_label'] = pd.Categorical(ww['quality_label'],
categories=['low', 'medium', 'high'])

rw['quality_label'] = rw['quality'].apply(lambda value: 'low'
if value <= 5 else 'medium'
if value <= 7 else 'high')

# here we are transforming these labels into categrical data type (specific to pandas) instead of simple string
rw['quality_label'] = pd.Categorical(rw['quality_label'],
categories=['low', 'medium', 'high'])

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
ww['encoded_quality_label'] = le.fit_transform(ww['quality_label'])  


In [ ]:
# Swap 0 for 3 to make the order make sense from low to high
ww['encoded_quality_label'].replace(0,3,inplace=True)
display(ww.head(20))

In [ ]:
ww.drop(['quality_label'], axis=1, inplace=True)

In [7]:
ww.drop(['quality'], axis=1, inplace=True)
display(ww.head())

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,encoded_quality_label
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,2
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,2
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,2
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,2
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,2


## Select the features to use as inputs

In [17]:
ww_select = ww[['fixed acidity', 'volatile acidity', 'chlorides', 'alcohol']]
display(ww_select.head())

,fixed acidity,volatile acidity,chlorides,alcohol
0,7.0,0.27,0.045,8.8
1,6.3,0.30,0.049,9.5
2,8.1,0.28,0.050,10.1
3,7.2,0.23,0.058,9.9
4,7.2,0.23,0.058,9.9


## Create column-vector for quality_label (y)

In [20]:
y_ww_quality = ww.encoded_quality_label
display(y_ww_quality.head())
display(y_ww_quality.shape)

0    2
1    2
2    2
3    2
4    2
Name: encoded_quality_label, dtype: int32

(4898,)

## Split into train and test data

In [44]:
from sklearn.model_selection import train_test_split
ww_features_train, ww_features_test, ww_quality_train, ww_quality_test = train_test_split(ww_select,y_ww_quality,test_size=0.2, random_state=19)

## Scale wine features to even out their impact on the model

In [45]:

# Also try different methods of normalization/standardization
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
ww_features_train_scaled = scaler.fit_transform(ww_features_train)
ww_features_test_scaled = scaler.transform(ww_features_test)

## Initialize and fit the model

In [25]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(max_iter=1000)
model.fit(ww_features_train_scaled, ww_quality_train)

LogisticRegression(max_iter=1000)

## Predict the test set

In [26]:
pred_model = model.predict(ww_features_test_scaled)

## Show accuracy score

In [38]:
from sklearn.metrics import accuracy_score
accuracy = ((accuracy_score(ww_quality_test, pred_model)) * 100).round(2)

print(f"{accuracy}% accuracy with prediction of low, medium, or high quality of a given wine, based on fixed acidity, volatile acidity, \nchlorides, and alcohol content.")


68.57

68.57% accuracy with prediction of low, medium, or high quality of a given wine, based on fixed acidity, volatile acidity, 
chlorides, and alcohol content.


## Show confusion matrix

In [39]:
from sklearn.metrics import confusion_matrix
display("Confusion matrix:")
display(confusion_matrix(ww_quality_test, pred_model))

'Confusion matrix:'

array([[162, 185,   0],
       [ 71, 510,   0],
       [  5,  47,   0]], dtype=int64)

In this case, the confusion matrix shows that the model never predicted high quality (third column)

## Create a function with several ML models at once

In [72]:
# Import the relevant models:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier


In [75]:
def models(X_train, y_train, X_test, y_test):
    
    log_reg = LogisticRegression(max_iter=1000)
    log_reg.fit(X_train, y_train)
    log_reg_train_predictions = log_reg.predict(X_train)
    log_reg_test_predictions = log_reg.predict(X_test)

    gauss = GaussianNB()
    gauss.fit(X_train, y_train)
    gauss_train_predictions = gauss.predict(X_train)
    gauss_test_predictions = gauss.predict(X_test)

    svc = SVC()
    svc.fit(X_train, y_train)
    svc_train_predictions = svc.predict(X_train)
    svc_test_predictions = svc.predict(X_test)

    svc_lin = SVC(kernel = 'linear', random_state = 0)
    svc_lin.fit(X_train, y_train)
    svc_lin_train_predictions = svc_lin.predict(X_train)
    svc_lin_test_predictions = svc_lin.predict(X_test)

    tree = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
    tree.fit(X_train, y_train)
    tree_train_predictions = tree.predict(X_train)
    tree_test_predictions = tree.predict(X_test)

    tree_gini = DecisionTreeClassifier(criterion = 'gini', random_state = 0)
    tree_gini.fit(X_train, y_train)
    tree_gini_train_predictions = tree_gini.predict(X_train)
    tree_gini_test_predictions = tree_gini.predict(X_test)

    forest = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 0)
    forest.fit(X_train, y_train)
    forest_train_predictions = forest.predict(X_train)
    forest_test_predictions = forest.predict(X_test)

    forest_gini = RandomForestClassifier(n_estimators = 100, criterion = 'gini', random_state = 0)
    forest_gini.fit(X_train, y_train)
    forest_gini_train_predictions = forest_gini.predict(X_train)
    forest_gini_test_predictions = forest_gini.predict(X_test)

    mlp = MLPClassifier(hidden_layer_sizes=(100,), max_iter=1000, learning_rate_init=0.001, early_stopping=True, activation='relu', solver='adam', random_state=0)
    mlp.fit(X_train, y_train)
    mlp_train_predictions = mlp.predict(X_train)
    mlp_test_predictions = mlp.predict(X_test)

    knn = KNeighborsClassifier()
    knn.fit(X_train, y_train)
    knn_train_predictions = knn.predict(X_train)
    knn_test_predictions = knn.predict(X_test)

    print('Logistic Regression Training Accuracy:', ((accuracy_score(y_train, log_reg_train_predictions)) * 100).round(2))
    print('Logistic Regression Test Accuracy:', ((accuracy_score(y_test, log_reg_test_predictions)) * 100).round(2))
    display(confusion_matrix(y_test, log_reg_test_predictions))
    print('Gaussian Naive Bayes Training Accuracy:', ((accuracy_score(y_train, gauss_train_predictions)) * 100).round(2))
    print('Gaussian Naive Bayes Test Accuracy:', ((accuracy_score(y_test, gauss_test_predictions)) * 100).round(2))
    display(confusion_matrix(y_test, gauss_test_predictions))
    print('Support Vector Classification Training Accuracy:', ((accuracy_score(y_train, svc_train_predictions)) * 100).round(2))
    print('Support Vector Classification Test Accuracy:', ((accuracy_score(y_test, svc_test_predictions)) * 100).round(2))
    display(confusion_matrix(y_test, svc_test_predictions))
    print('Linear Support Vector Classification Training Accuracy:', ((accuracy_score(y_train, svc_lin_train_predictions)) * 100).round(2))
    print('Linear Support Vector Classification Test Accuracy:', ((accuracy_score(y_test, svc_lin_test_predictions)) * 100).round(2))
    display(confusion_matrix(y_test, svc_lin_test_predictions))
    print('Decision Tree Classifier Training Accuracy:', ((accuracy_score(y_train, tree_train_predictions)) * 100).round(2))
    print('Decision Tree Classifier Test Accuracy:', ((accuracy_score(y_test, tree_test_predictions)) * 100).round(2))
    display(confusion_matrix(y_test, tree_test_predictions))
    print('Gini Decision Tree Classifier Training Accuracy:', ((accuracy_score(y_train, tree_gini_train_predictions)) * 100).round(2))
    print('Gini Decision Tree Classifier Test Accuracy:', ((accuracy_score(y_test, tree_gini_test_predictions)) * 100).round(2))
    display(confusion_matrix(y_test, tree_gini_test_predictions))
    print('Forest Training Accuracy:', ((accuracy_score(y_train, forest_train_predictions)) * 100).round(2))
    print('Forest Test Accuracy:', ((accuracy_score(y_test, forest_test_predictions)) * 100).round(2))
    display(confusion_matrix(y_test, forest_test_predictions))
    print('Gini Forest Training Accuracy:', ((accuracy_score(y_train, forest_gini_train_predictions)) * 100).round(2))
    print('Gini Forest Test Accuracy:', ((accuracy_score(y_test, forest_gini_test_predictions)) * 100).round(2))
    display(confusion_matrix(y_test, forest_gini_test_predictions))
    print('Neural Network MLP Classifier Training Accuracy:', ((accuracy_score(y_train, mlp_train_predictions)) * 100).round(2))
    print('Neural Network MLP Classifier Test Accuracy:', ((accuracy_score(y_test, mlp_test_predictions)) * 100).round(2))
    display(confusion_matrix(y_test, mlp_test_predictions))
    print('K Nearest Neighbors Classifier Training Accuracy:', ((accuracy_score(y_train, knn_train_predictions)) * 100).round(2))
    print('K Nearest Neighbors Classifier Test Accuracy:', ((accuracy_score(y_test, knn_test_predictions)) * 100).round(2))
    display(confusion_matrix(y_test, knn_test_predictions))

Call the big function on white wine features (input) quality (output)

In [76]:
models(ww_features_train_scaled, ww_quality_train, ww_features_test_scaled, ww_quality_test)

Logistic Regression Training Accuracy: 70.09
Logistic Regression Test Accuracy: 71.43


array([[146, 146,   0],
       [ 92, 554,   0],
       [ 10,  32,   0]], dtype=int64)

Gaussian Naive Bayes Training Accuracy: 67.71
Gaussian Naive Bayes Test Accuracy: 70.92


array([[102, 188,   2],
       [ 52, 591,   3],
       [  1,  39,   2]], dtype=int64)

Support Vector Classification Training Accuracy: 72.21
Support Vector Classification Test Accuracy: 73.27


...

array([[192,  99,   1],
       [ 75, 569,   2],
       [  5,  18,  19]], dtype=int64)

Neural Network MLP Classifier Training Accuracy: 71.44
Neural Network MLP Classifier Test Accuracy: 72.24


array([[145, 147,   0],
       [ 83, 563,   0],
       [  1,  41,   0]], dtype=int64)

K Nearest Neighbors Classifier Training Accuracy: 79.53
K Nearest Neighbors Classifier Test Accuracy: 68.57


array([[155, 137,   0],
       [133, 510,   3],
       [  6,  29,   7]], dtype=int64)